1. Предобработать данные (пропуски, замена текста, дубликаты и т.д)
2. Использовать для классификации sklearn.ensemble.RandomForestClassifier
3. Подобрать лучшие гиперпараметры (n_estimators, criterion, max_depth)
4. Провести кроссвалидацию модели
5. Оценить метрики бинарной классификации (Precision, recall, f1)
6. Построить ROC-кривую для оценки качества классификатора

In [32]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve,auc
from sklearn.model_selection import GridSearchCV
from plotly import graph_objects as go


In [33]:
df_titanic = pd.read_csv('School_DA/ML_for_DA/Lesson2/titanicdf.csv')
df_work = df_titanic.copy()
df_titanic.head(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [34]:
for i in (df_work.drop(columns=['PassengerId', 'Name', 'Ticket', 'Age', 'Fare', 'Cabin'])).columns:
    print(i)
    print(df_work[i].value_counts())
    print('')


Survived
0    549
1    342
Name: Survived, dtype: int64

Pclass
3    491
1    216
2    184
Name: Pclass, dtype: int64

Sex
male      577
female    314
Name: Sex, dtype: int64

SibSp
0    608
1    209
2     28
4     18
3     16
8      7
5      5
Name: SibSp, dtype: int64

Parch
0    678
1    118
2     80
5      5
3      5
4      4
6      1
Name: Parch, dtype: int64

Embarked
S    644
C    168
Q     77
Name: Embarked, dtype: int64



In [35]:
# удаляем колонки, из которых точно не получим фичи, так как невозможно разбить на категории
df_work = df_work.drop(columns=['PassengerId', 'Fare', 'Cabin', 'Embarked'])
# переводим пол в булевое значение
df_work['male'] = df_work.Sex.apply(lambda x: 1 if x == 'male' else 0)
# удаляем колонку пола пассажира строкового типа, так как это уже более неактуально
df_work = df_work.drop(columns=['Sex'])

In [36]:
# df_work.Embarked.value_counts()/len(df_work)
# разбиваем на категории класс пассажиров
df_work = pd.get_dummies(data=df_work, columns=['Pclass'])

In [37]:
# разбиваем на возрастные группы
df_work['18-'] = df_work.Age.apply(lambda x: 1 if x < 18 else 0)
df_work['18-35'] = df_work.Age.apply(lambda x: 1 if ((x >= 18) & (x < 36)) else 0)
df_work['36-45'] = df_work.Age.apply(lambda x: 1 if ((x >= 36) & (x < 46)) else 0)
df_work['46-60'] = df_work.Age.apply(lambda x: 1 if ((x >= 46) & (x < 60)) else 0)
df_work['60+'] = df_work.Age.apply(lambda x: 1 if x >= 60 else 0)
# удаляем колонку с возрастом пассажиров
df_work = df_work.drop(columns='Age')

In [38]:
# заметил, что проводить анализ ФИО не самая лучшая идея, т.к. все семьи или друзья, покупали билеты разом и имеют один
# и тот же номер билета, поэтому разбиваем билеты на группы таким образом, чтобы каждый билет принадлежал группе, если
# один билет куплен на нескольких человек
df_ticket = (df_titanic.groupby(by='Ticket')[['Ticket']].count()).rename(columns={'Ticket':'quantity_ticket'}).\
    reset_index()
df_ticket = df_work.groupby(by='Ticket')[['Ticket']].count().rename(columns={'Ticket':'quantity_ticket'}).\
    sort_values(by='quantity_ticket', ascending=False).reset_index()
df_ticket = df_ticket.reset_index().rename(columns={'index':'ticket_group'})
# df_ticket.ticket_group = df_ticket.ticket_group.apply(lambda x: x+1)
# df_ticket.ticket_group = pd.concat([df_ticket[df_ticket.quantity_ticket > 1]['ticket_group'],
#                                     df_ticket[df_ticket.quantity_ticket == 1].quantity_ticket.apply(lambda x: 0)])
df_ticket = df_ticket[['Ticket', 'ticket_group']]
df_work = df_work.merge(df_ticket, on='Ticket')
df_work = df_work.drop(columns='Ticket')
df_work

,Survived,Name,SibSp,Parch,male,Pclass_1,Pclass_2,Pclass_3,18-,18-35,36-45,46-60,60+,ticket_group
0,0,"Braund, Mr. Owen Harris",1,0,1,0,0,1,0,1,0,0,0,163
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,0,0,1,0,0,0,0,1,0,0,382
2,1,"Heikkinen, Miss. Laina",0,0,0,0,0,1,0,1,0,0,0,310
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,0,0,1,0,0,0,1,0,0,0,117
4,0,"Futrelle, Mr. Jacques Heath",1,0,1,1,0,0,0,0,1,0,0,117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,"Sutehall, Mr. Henry Jr",0,0,1,0,0,1,0,1,0,0,0,329
887,0,"Montvila, Rev. Juozas",0,0,1,0,1,0,0,1,0,0,0,487
888,1,"Graham, Miss. Margaret Edith",0,0,0,1,0,0,0,1,0,0,0,370
889,1,"Behr, Mr. Karl Howell",0,0,1,1,0,0,0,1,0,0,0,291


In [39]:
# формируем тестовые и тренировочные выборки
X_train, X_test, y_train, y_test = train_test_split(df_work.drop(['Survived', 'Name'], axis=1),
                                                    df_work['Survived'],
                                                    train_size=0.75,
                                                    random_state=42)
# объявляем классификатор
clf = RandomForestClassifier()
model = clf.fit(X_train, y_train)
clas_predict = model.predict(X_test)
print('Accuracy:',accuracy_score(y_test, clas_predict))
print('Precision:',precision_score(y_test, clas_predict))
print('Recall:',recall_score(y_test, clas_predict))
print('F1:',f1_score(y_test, clas_predict))

Accuracy: 0.7847533632286996
Precision: 0.6875
Recall: 0.7857142857142857
F1: 0.7333333333333334


In [40]:
# подбираем параметры n_estimators, criterion, max_depth
param_dict = {
    'n_estimators': [100, 200, 300],
    'max_depth' : [5, 6, 7],
    'criterion' :['gini', 'entropy']
}

In [41]:
# выводим лучшее сочетание параметров
rfc = RandomForestClassifier(random_state=42)
clf_gscv = GridSearchCV(estimator=rfc, param_grid=param_dict, cv=5)
print(clf_gscv.fit(X_train, y_train))
clf_gscv.best_params_

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 6, 7],
                         'n_estimators': [100, 200, 300]})


{'criterion': 'entropy', 'max_depth': 6, 'n_estimators': 200}

In [42]:
# получаем показатели метрик качества с лучшими параметрами
rfc_best = RandomForestClassifier(random_state=42, criterion='entropy', max_depth=6, n_estimators=200)
rfc_best.fit(X_train, y_train)
rfc_best_predict = rfc_best.predict(X_test)
print('Accuracy:',accuracy_score(y_test, rfc_best_predict))
print('Precision:',precision_score(y_test, rfc_best_predict))
print('Recall:',recall_score(y_test, rfc_best_predict))
print('F1:',f1_score(y_test, rfc_best_predict))

Accuracy: 0.8161434977578476
Precision: 0.7590361445783133
Recall: 0.75
F1: 0.7544910179640718


In [46]:
# получаем показатель ROC AUC
pred_prob = rfc_best.predict_proba(X_test)[:,1]
false_positive_rate, true_positive_rate, treshold = roc_curve(y_test, pred_prob)
tree_auc = roc_auc_score(y_test, pred_prob)
print('RandomForestClassifier: ROC AUC=%.3f' % (tree_auc))


RandomForestClassifier: ROC AUC=0.867


In [67]:
# строим кривую ROC AUC
roc_auc = auc(false_positive_rate, true_positive_rate)
fig = go.Figure()
fig.add_trace(go.Scatter(x=false_positive_rate.tolist(),
                         y=true_positive_rate.tolist(),
                         name='ROC AUC ' + str(round(tree_auc, 2)))
                         )
fig.add_trace(go.Scatter(x=np.linspace(0,1,2), y=np.linspace(0,1,2), line=dict(dash='dot'), name='Random Classifier'))
fig.update_layout(xaxis_title='False Positive Rate', yaxis_title='True Positive Rate')
fig.show()
